Investigate whether the QSO are handled correctly  : if the values calculated for a given lightcurve are actually carried on with that lightcurve 
    

In [3]:

import numpy as np
import os

def load_sdss_qso(catalog = 'DB_QSO'):
    '''
    DB_QSO  : DB_QSO_S82.dat, with SDSS ugriz, for 9258 objects
    '''
   
    if catalog == 'DB_QSO' : 
        # note : ra and dec in degrees
        File = 'DB_QSO_S82.dat'
        datatable = np.genfromtxt(File)
        colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']
        data_quasars = {}
        print 'zipping quasars...'
        for label, column in zip(colnames, datatable.T):
            data_quasars[label] = column
        
        print 'I read in data for ', len(data_quasars['ra']), ' SDSS quasars'
        print 'From catalog ', File
        return data_quasars
              
def load_crts_qso(inDir):
    # load names of quasars, which already contain ra and dec infor 
    CRTS_qso  = os.listdir(inDir)
    print '\nI loaded names of ', len(CRTS_qso), ' CRTS quasars'
    return CRTS_qso

crts_dirs = ['../QSO_CRTS_processed_err_w/','../stars_CRTS_proc_err_w_good/']


In [41]:
def get_ra_dec_CRTS(ra_hms, dec_hms):
    """
    Extracting RA, DEC information from the QSO  name for the CRTS case 
    """
    
    dec_hms_split = np.empty(0, dtype=str)
    for i in range(0,len(dec_hms)):
        dec_hms_split = np.append(dec_hms_split,  dec_hms[i][0:3]+' '+dec_hms[i][3:5]+' '+dec_hms[i][5:9])
    
    
    ra_hms_split = np.empty(0, dtype=str)
    for i in range(0,len(ra_hms)):
        ra_hms_split = np.append(ra_hms_split, ra_hms[i][0:2]+' '+ ra_hms[i][2:4] + ' ' + ra_hms[i][4:9])
    return ra_hms_split, dec_hms_split
     
def HMS2deg(ra='', dec=''):
    """
    From http://www.bdnyc.org/2012/10/15/decimal-deg-to-hms/  
    Converting  ra and dec from h:m:s   and deg:m:s  to  degrees.decimal 
    I assume they are using ICRS coordinates 
    """
    RA, DEC, rs, ds = '', '', 1, 1
    if dec:
        D, M, S = [float(i) for i in dec.split()]
        if str(D)[0] == '-':
            ds, D = -1, abs(D)
        deg = D + (M/60) + (S/3600)
        DEC = '{0}'.format(deg*ds)
      
    if ra:
        H, M, S = [float(i) for i in ra.split()]
        if str(H)[0] == '-':
            rs, H = -1, abs(H)
        deg = (H*15) + (M/4) + (S/240)
        RA = '{0}'.format(deg*rs)
      
    if ra and dec:
        return (RA, DEC)
    else:
        return RA or DEC
    
    
def convert_to_deg(ra_split, dec_split):
    '''
    Converts ra and dec from h:m:s  extracted from the quasar name with 
    get_ra_dec_CRTS()  to  degrees.decimal , using HMS2deg() function
    '''
    ra_deg  =   np.empty(0, dtype=float)
    dec_deg =   np.empty(0, dtype=float)
    
    for i in range(0,len(dec_split)):
        dec_deg = np.append(dec_deg,float(HMS2deg(dec=dec_split[i])))
    
    
    for i in range(0,len(ra_split)):
        ra_deg = np.append(ra_deg,float(HMS2deg(ra=ra_split[i])))
    return ra_deg , dec_deg
    

def match_catalogs(cat1_ra, cat1_dec, cat2_ra, cat2_dec):
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    cat1 = SkyCoord(ra=cat1_ra*u.degree, dec=cat1_dec*u.degree)
    cat2 = SkyCoord(ra=cat2_ra*u.degree, dec=cat2_dec*u.degree)
    idx, sep2d, dist3d = cat1.match_to_catalog_sky(cat2)
    #np.savez(archive_file, SDSS_matching_rows = idx, matched_radius=sep2d ) 
    return idx, sep2d 


In [39]:
# load names from CRTS 
DIR = crts_dirs[0]
crts_qso_names = load_crts_qso(DIR)

# load data from SDSS
sdss_qso_data =  load_sdss_qso()



I loaded names of  7601  CRTS quasars
zipping quasars...
I read in data for  9258  SDSS quasars
From catalog  DB_QSO_S82.dat


In [11]:
length= len(crts_qso_names)
print '- Computing average mag, err , extracting ra, dec for %i points' % length

avg_mag=[]
avg_err=[]
ra_ls =[]
dec_ls=[]
mjd_span = []
mjd_uniq_N = []
N_rows = []        
c=0
for i in range(length):
    file = str(crts_qso_names[i])
    #print '\nCRTS quasars file ',i, 'out of',  length
    mjd,flx4,err = np.loadtxt(DIR+'%s' % (file),usecols=(0,1,2),unpack=True)

    # 1) Average brightness per LC
    avg_mag.append(np.mean(flx4))

    # 2) Average error per LC
    avg_err.append(np.mean(err))

    # 3) Delta_MJD : span in days between final and first day
    mjd_span.append(int(mjd.max()-mjd.min()))  

    # 4) N_MJD : number of days (i.e. taking the integer part of mjd 
    # date and picking only those that are unique)
    unique_mjds = np.unique([int(day) for day in mjd])
    mjd_uniq_N.append(len(unique_mjds))  # number of days observed 

    # 5) N_rows : number of rows per LC 
    N_rows.append(len(mjd))

    # 6) CRTS  ra and dec for that object ( no  need to pull ra, dec 
    # from a separate file, matching by name, because the qso name
    # already includes that... )
    ra_ls.append(file[4:13])
    dec_ls.append(file[13:-4])
    c += 1 
    if c % 5 == 0:
            pers = (100.0*c) / float(length)
            print('\r----- Already read %d%% of QSO '%pers), 

- Computing average mag, err , extracting ra, dec for 7601 points
----- Already read 99% of QSO 


In [14]:
avg_err[0]

0.053901886792452829

In [16]:
crts_qso_names[0]

'out_001053.56+000642.9.txt'

In [17]:
ra_ls[0]

'001053.56'

In [18]:
dec_ls[0]

'+000642.9'

In [19]:
# Split CRTS  ra, dec from hms to h m s 
ra_hms_split, dec_hms_split = get_ra_dec_CRTS(ra_ls, dec_ls)
# Convert CRTS  ra, dec from hms to deg  
ra_deg_CRTS, dec_deg_CRTS = convert_to_deg(ra_hms_split, dec_hms_split)


In [21]:
dec_hms_split

array(['+00 06 42.9', '-00 31 33.2', '-00 18 30.1', ..., '+00 03 29.8',
       '+00 32 27.8', '+00 30 56.4'], 
      dtype='|S11')

In [22]:
ra_deg_CRTS

array([   2.72316667,    2.71229167,    2.69995833, ...,  359.41179167,
        359.40816667,  359.401125  ])

In [23]:
dec_deg_CRTS

array([ 0.11191667, -0.52588889, -0.30836111, ...,  0.05827778,
        0.54105556,  0.51566667])

In [38]:
#from astropy.coordinates import SkyCoord
#from astropy import units as u
#ra_deg = []
#dec_deg = [] 
#for i in range(len(ra_hms_split)): 
#    a = SkyCoord(ra_hms_split[i]+dec_hms_split[i], unit=(u.hourangle, u.deg))
#    ra_deg.append(a.ra.deg)
#    dec_deg.append(a.dec.deg)

In [42]:
print '\n- Computing the SDSS matching rows to CRTS quasars'
SDSS_matching_rows , matched_radius= match_catalogs(cat1_ra=ra_deg_CRTS, 
                                                    cat1_dec=dec_deg_CRTS, 
                                                    cat2_ra= sdss_qso_data['ra'], 
                                                    cat2_dec=sdss_qso_data['dec']) 


- Computing the SDSS matching rows to CRTS quasars


In [43]:
crts_qso_names_radec = [name[4:-4] for name in crts_qso_names] 

In [44]:
crts_qso_names_radec[0]

'001053.56+000642.9'

In [45]:
crts_qso_names[0]

'out_001053.56+000642.9.txt'

In [54]:
ind=SDSS_matching_rows

In [55]:
datatable=np.array([crts_qso_names_radec, avg_mag, avg_err, sdss_qso_data['z'][ind], 
                      sdss_qso_data['i'][ind], sdss_qso_data['r'][ind],
                      sdss_qso_data['g'][ind], sdss_qso_data['u'][ind], 
                      sdss_qso_data['redshift'][ind], sdss_qso_data['ra'][ind], 
                      sdss_qso_data['dec'][ind], ra_deg_CRTS, dec_deg_CRTS,
                      mjd_span, mjd_uniq_N, N_rows])
colnames = ['CRTS_id','CRTS_avg_mag','CRTS_avg_err', 'z', 'i', 'r', 'g', 'u', 
                'redshift', 'ra_SDSS', 'dec_SDSS', 'ra_CRTS', 'dec_CRTS', 
                'mjd_span', 'mjd_uniq_N', 'N_rows']

In [56]:
data_qso_SDSS_CRTS= {}
print '\nZipping  quasars...'

for label, column in zip(colnames, datatable):
    data_qso_SDSS_CRTS[label] = column


Zipping  quasars...


In [58]:
data_qso_SDSS_CRTS['CRTS_id'][0]

'001053.56+000642.9'

In [62]:
data_qso_SDSS_CRTS['CRTS_avg_mag'][0]

'19.4385698113'

In [67]:
keys = colnames
DATA = np.column_stack((datatable))    

header=''
for key in keys : 
    header= header+'{:<10}'.format(key[:10])+' '

catalog='DB_QSO'
archive_SDSS_CRTS_qso = 'CRTS_SDSS_cross_matched_qso_'+catalog+'_catalog.txt'
#fmt = ['%s', '%.4e', '%10.5f']
# old fmt '%11.5f'*12+'%6.i'+'%5.i'*2
np.savetxt(archive_SDSS_CRTS_qso, DATA, delimiter =' ', fmt='%s  '*16, header=header)

